## Assess permanent forest cover in headwater and reference watersheds

In [1]:
from rasterstats import zonal_stats
import fiona
import geopandas as gpd
import os
import pandas as pd
import numpy as np

import itertools
import multiprocess

home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"

#### Compare permanent forest cover in headwater catchments identified using NLCD vs LCMAP 

In [2]:
# all headwaters shapefile 
shp = os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sbr_shp.shp")

In [3]:
# NLCD: permanent forest 100% and the catchment is greater than 0.025 (>= 0.025 gives at least 5 ecostress pixels to a headwater catchment)
shape_gdf = gpd.read_file(shp)
perm_forest_pd = pd.read_csv(os.path.join(home, "Data", "nlcd_permanent_forest", "headwater_pct_forest.csv"))
nlcd_pf = shape_gdf.merge(perm_forest_pd[['percent_forest', 'NHDPlusID']], on = "NHDPlusID", how = 'left')
nlcd_pf = nlcd_pf[nlcd_pf['percent_forest'] == 1]
nlcd_pf = nlcd_pf[nlcd_pf.AreaSqKm_x >= 0.025]
nlcd_pf.shape

(21271, 6)

In [4]:
# compare with LCMAP permanent forest 
shape_gdf = gpd.read_file(shp)
lcmap = pd.read_csv(os.path.join(home, "Data", "lcmap_permanent_forest", "lcmap_headwater_pct_forest.csv"))
lcmap['percent_forest'] = lcmap['36.0'] / lcmap.iloc[:,1:(lcmap.shape[1]-1)].sum(axis=1)
lcmap_pf = shape_gdf.merge(lcmap[['percent_forest', 'NHDPlusID']], on = "NHDPlusID", how = 'left')
lcmap_all = lcmap_pf
lcmap_pf = lcmap_pf[lcmap_pf['percent_forest'] == 1]
#lcmap_pf = lcmap_pf[lcmap_pf['percent_forest'] >= 0.98]
lcmap_pf = lcmap_pf[lcmap_pf.AreaSqKm_x >= 0.025]
lcmap_pf.shape

lcmap_all.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_lcmap.shp"))

(30045, 6)

In [7]:
# we trust the lcmap data more and i have it for more dates 
# it also gives us more catchments to work with, which is nice 
nlcd_ids = nlcd_pf['NHDPlusID'].to_numpy()
lcmap_ids = lcmap_pf['NHDPlusID'].to_numpy()

# find which nlcd ids are in lcmap 
s = set(lcmap_ids)
booleans = [i in s for i in nlcd_ids]
nlcd_in_lcmap = nlcd_ids[booleans] 
print('nlcd in lcmap = ', len(nlcd_in_lcmap))

# find which nlcd ids are not in lcmap  
s = set(lcmap_ids)
booleans = [i not in s for i in nlcd_ids]
nlcd_notin_lcmap = nlcd_ids[booleans] # 18865
print('nlcd NOT in lcmap = ', len(nlcd_notin_lcmap))


# find which lcmap are not in nlcd 
s = set(nlcd_ids)
booleans = [i not in s for i in lcmap_ids]
lcmap_notin_nlcd = lcmap_ids[booleans]
print('lcmap NOT in nlcd = ', len(lcmap_notin_nlcd))


nlcd in lcmap =  16777
nlcd NOT in lcmap =  4494
lcmap NOT in nlcd =  13268


In [9]:
# I am going to use the lcmap 100% forested headwater catchments that are >= 0.025 km2 
lcmap_pf.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_perm_forest.shp"))

# reproject and save 
lcmap_pf_32617 = lcmap_pf.to_crs("EPSG:32617")
lcmap_pf_32617.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_perm_forest_32617.shp"))

# get the mean area of all catchments
np.mean(lcmap_pf_32617.AreaSqKm_x)

/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/6_/17bx0qz13sjdf13yymjj7kb00000gn/T/ipykernel_88480/3447724396.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  lcmap_pf.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_perm_forest.shp"))


### Permanent forest cover in the refrence watersheds

In [8]:
shp = os.path.join(home, "Data", "Catchments", "Reference", "gages_ii",  "reference_keep_4326.shp")
tif = os.path.join(home, "Data", "lcmap_permanent_forest", "lcmap_permanent_forest.tif")
lcmap_rw = gpd.read_file(os.path.join(home, "Data", "lcmap_permanent_forest", "lcmap_reference_watersheds.csv"))
lcmap_rw = lcmap_rw.rename(columns=lcmap_rw.iloc[0])
lcmap_rw = lcmap_rw.iloc[1:,1:lcmap_rw.shape[1]-1]
cols = lcmap_rw.columns[lcmap_rw.dtypes.eq('object')]
lcmap_rw[cols] = lcmap_rw[cols].apply(pd.to_numeric, errors='coerce')
lcmap_rw = lcmap_rw.replace(np.nan, 0)
lcmap_rw['percent_forest'] = lcmap_rw['36.0'] / lcmap_rw.iloc[:,0:(lcmap_rw.shape[1]-1)].sum(axis=1)
lcmap_rw[['GAGE_ID', 'percent_forest']].to_csv(os.path.join(home, "Data", "lcmap_permanent_forest", "lcmap_perm_forest_reference_watersheds.csv"), index=False)